看 PyEcharts 名字就猜得到



    PyEcharts = Python + Echarts



Echarts 是一个由百度开源的数据可视化工具，凭借着良好的交互性，精巧的图表设计，得到了众多开发者的认可，而 Python 就不用多说了。



当 Python 遇到了 Echarts，就变成了 PyEcharts。

In [3]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib inline
from datetime import datetime

In [4]:
from pyecharts.charts import Line, Kline, Pie, Grid,Timeline, WordCloud

- Line 用来画线

- Kline 用来画 K 线

- Pie 用来画饼

- Grid 用来画多个坐标系

- Overlap 用来叠加不同对象 (比如 Line 和 Kline)

- Timeline 用来画时间轴轮播图

- WordCloud 主要展示词云图

# K线图

K 线又称蜡烛图，阴阳线。原来是日本米商用于记录米市中的行情波动的，后因其表示价格趋势的方法直观，真实，逐渐成为使用最广泛的技术图形。图像如下：

![](./img/K线图.png)

颜色不是最重要的，看 A 股的 K 线记住是「红涨绿跌」，看美股的 K 线记住是「绿涨红跌」。

重要的是 K 线是由很多「蜡烛」状的单元责成，每个单位包含四个数据：即开盘价、最高价、最低价、收盘价。所有的 K 线都是围绕这四个数据展开，反映大势的状况和价格信息。

数据的描述如下



- 起始日：2016-01-01

- 终止日：2019-05-13

- 四个外汇：欧元美元、美元日元、美元人民币，英镑美元

- 三个加密货币：比特币、以太币、瑞波币

其中



- 货币用的不是市场常见格式，比如「欧元美元」用 EURUSD=X，而不是 EURUSD，而「美元日元」用 JPY=X 而不是 USDJPY



- 加密货币都是以美元计价，用的格式是「比特币」用 BTC-USD，「以太币」用 ETH-USD,「瑞波币」用 XRP-USD



In [5]:
from yahoofinancials import YahooFinancials

In [6]:
start_date = '2016-01-01'
end_date = '2019-05-13'

currencies = ['EURUSD=X', 'JPY=X', 'CNY=X', 'GBPUSD=X']
cryptocurrencies = ['BTC-USD', 'ETH-USD', 'XRP-USD']

In [ ]:
FX_obj = YahooFinancials(currencies)
CRX_obj = YahooFinancials(cryptocurrencies)
FX_daily = FX_obj.get_historical_price_data(start_date, end_date, 'daily')
CFX_daily = CRX_obj.get_historical_price_data(start_date, end_date, 'daily')

In [ ]:
# 将原始数据转换成DataFrame
def data_converter(price_data, code, asset):
    # convert raw data to dataframe
    if asset == 'FX':
        code = str(code[3:] if code[:3] == 'USD' else code) + '=X'

    columns = ['open', 'close', 'low', 'high']
    price_dict = price_data[code]['prices']
    index = [p['formatted_date'] for p in price_dict]
    price = [[p[c] for c in columns] for p in price_dict]

    data = pd.DataFrame(
        price,
        index=pd.Index(index, name='date'),
        columns=pd.Index(columns, name='OHLC'))
    return data

第 3 行完全是为了 YahooFinancial 里面的输入格式准备的。如果 Asset 是加密货币，直接用其股票代码；如果 Asset 是汇率，一般参数写成 EURUSD 或 USDJPY



如果是 EURUSD，转换成 EURUSD=X

如果是 USDJPY，转换成 JPY=X



第 6 行定义好开盘价、收盘价、最低价和最高价的标签。



第 7 行获取出一个「字典」格式的数据。



第 8, 9 行用列表解析式 (list comprehension) 将日期和价格获取出来。



第 11 到 13 行定义一个 DataFrame



值为第 9 行得到的 price 列表

行标签为第 8 行得到的 index 列表

列标签为第 6 行定义好的 columns 列表

In [ ]:
curr = 'USDCNY'
data = data_converter(FX_daily, curr, 'FX')
data.head(3).append(data.tail(3))

## Matplotlib图

Matplotlib 里面没有直接画 K 线的方法，我们用里面的块 (patch) 对象，对其填充上色，空心红色代表阳线，实心深青色代表阴线

In [ ]:
def mpl_Kline(curr, data):

    fig = plt.figure(figsize=(16, 6), dpi=100)
    ax = fig.add_axes([0, 1, 1, 1])
    ax.set_title(curr + '- K Line', fontsize=15)
    ax.set_xlim(-1, len(data) + 1)

    for i in range(len(data)):
        open, close, high, low = data.iloc[i, 0], data.iloc[i, 1], data.iloc[
            i, 3], data.iloc[i, 2]

        if close > open:
            ax.add_patch(
                patches.Rectangle((i - 0.2, open),
                                  0.4,
                                  close - open,
                                  fill=False,
                                  color=r_hex))
            ax.plot([i, i], [low, open], r_hex)
            ax.plot([i, i], [close, high], r_hex)
        else:
            ax.add_patch(
                patches.Rectangle((i - 0.2, close),
                                  0.4,
                                  open - close,
                                  color=dt_hex))
            ax.plot([i, i], [low, high], color=dt_hex)

    ax.set_xticks(range(0, len(data), 10))
    ax.set_xticklabels([data.index[i] for i in ax.get_xticks()], rotation=90)

    return ax

第 3-6 行设置了图的大小、dpi、坐标系、标题和 x 轴范围。




第 8-17 行是核心



第 8-9 行：对每一个数据，获取出开盘价 (open)、收盘价 (close)、最高价 (high)、最低价 (low)。



第 11-14 行：如果收盘价 > 开盘价 (阳线)



用 add_patch 方法加一个空心红色块 (patches.Rectangle)，确定红块的左下点 x-y 坐标 (i-0,2, open)，宽为 0.4，高为 close 和 open 之差。此外 fill 设为 False 就是画个空心矩形。



用 plot 方法加两条线，横坐标都是 i，第一条线纵坐标从 low 到 open，第一条线纵坐标从 close 到 high。







第 15-17 行：如果收盘价 < 开盘价 (阴线)



用 add_patch 方法加一个实心深青色块 (patches.Rectangle)，确定深青块的左下点 x-y 坐标 (i-0,2, close)，宽为 0.4，高为 open 和 close 之差。此外 fill 设为 True 就是画个实心矩形。



用 plot 方法加一条线，横坐标都是 i，纵坐标从 low 到 high (为什么画一条线呢？矩形是实心的啊)






第 19-20 行设置了 x 轴刻度以及标签，并把日期标签旋转 90 度，以免标签相互重叠。

In [ ]:
mpl_Kline(curr, data)

## PyEcharts图

PyEcharts 里画 K 线用到 Kline 对象，除此之外我们添加最高价和最低价两条线 Line 对象，再用 Overlap 对象来「叠加」它们

In [ ]:
date = data.index
price = data.values

kline = Kline(curr + 'Chart', title_pos='center')
kline.add(
    'K - Line ',
    date,
    price,
    tooltip_tragger='axis',
    is_datazoom_show=True,
    legend_pos='right',
    legend_orient='vertiacal',
    legend_text_size=10)

line2 = Line()
features = ['high', 'low']

for feature in featurnes:
    line2.add(feature, date, data[feature], tooltip_tragger='axis')

overlap = Overlap(width=1000, height=400)
overlap.add(kline)
overlap.add(line2)
# overlap.render(path=u'USDCNY Chart.html')

第 1-2 行获取日期和汇率。



第 4 行创建 K 线对象 Kline，设置好标题 "xxx Chart" 和位置 center。第 5-6 行在 Kline 上添加属性



图例： 'K-Line'，

x 坐标轴数据：日期

y 坐标轴数据：一定要按 [开盘值, 收盘值, 最低值, 最高值] 的顺序，之前处理数据特意按这个顺序设定 DataFrame 的列标签的

x 坐标轴可拉伸：True

图例位置：右边

图例排序：竖直

图例文字大小：10



第 8 行创建折线对象 Line。第 9-13 行在 Line 上添加两条折线，一条是最高价，一条是最低价。



第 14 行创建叠加对象 Overlap。第 15-16 行在 Overlap 上分别添加之前的 Kline 和 Line，这样就把所有对象整合在一起了。



第 17 行如果被运行，该动态图被生成到 USDCNY Chart.html 网页文件里；如果没被运行，该动态图将显示在 Jupyter Notebook 中。

本例用到了 pyecharts 中的三个原件：Kline，Line 和 Overlap。它们的用法都是先创建 (可以带些必要属性，比如标题和尺寸)，再用 add 方法添加额外属性。将其流程通用化得到



    object = Object( 必要属性 )

    object.add( 额外属性 )

如果你看懂了以上内容，你可以学会了 PyEcharts。接下来的所有例子都万变不离其宗。后面用 PyEcharts 画出三个最常见的交易策略要看的指标图，分别是



1.2 节的移动均线图 (Moving Average, MA)

1.3 节的布林带 (Bollinger Band, BBand)

1.4 节的相对强弱指标 (Relative Strength Index, RSI)



MA, BBand 和 RSI 是量化交易常用的技术指标，通常我们用 talib 包里的函数直接计算。



注:为节省学习时间,本人对涉及量化交易画图部分进行了省略,若感兴趣可自行点击下面链接参考学习

[移动均线图,布林带,相对强弱指标](https://mp.weixin.qq.com/s?__biz=MzIzMjY0MjE1MA==&mid=2247487057&idx=1&sn=265191abb333e69e65a243f2ed4cdf71&chksm=e8908d58dfe7044ec93d24cc155911d65bc64cf1504ab24e7fab7f3b32bd9f19efb107c2d84b&scene=21#wechat_redirect)

# PyEcharts多图

## 网格(Grid)图

### 数据准备

In [ ]:
stock_data = pd.read_csv(
    './data/data for pyecharts/1Y_Stock_Data.csv',
    parse_dates=[0],
    dayfirst=True)
stock_data.head().append(stock_data.tail())

In [ ]:
data = pd.read_csv(
    './data/data for pyecharts/S&P500.csv',
    index_col=0,
    parse_dates=True,
    dayfirst=True)
spx = data[['Adj Close']].loc['2018-02-26':'2019-02-26']
spx.head(3).append(spx.tail(3))

### 绘图

In [ ]:
code = 'AAPL'
stock = stock_data[stock_data['Symbol'] == code]

date = stock['Date'].dt.strftime('%d-%b-%Y')
price = stock[['Open', 'Close', 'Low', 'High']].values

kline = Kline(code, title_pos='left')
kline.add('', date, price, tooltip_tragger='axis', is_datazoom_show=True)

line1 = Line('SPX', title_top='55%')
line1.add(
    '',
    date,
    spx.values,
    yaxis_min=2200,
    yaxis_max=3000,
    mark_point=['min'],
    is_datazoom_show=True,
    datazoom_xaxis_index=[2, 1, 0])

line2 = line('VIX', title_top='75%')
line2.add(
    '',
    date,
    vix.values,
    yaxis_min=0,
    yaxis_max=40,
    mark_point=['max'],
    is_datazoom_show=True,
    datazoom_xaxis_index=[2, 1, 0])

grid = Grid(width=1000, height=600)
grid.add(line2, grid_top='75%')
grid.add(line1, grid_top='55%', grid_bottom='30%')
grid.add(kline, grid_top='5%', grid_bottom='50%')
# grid.render(path='AAPL&BIX.html')

第 1-5 行用 code 获取股票数据，并获取日期和价格。为了画 K 线，价格数组的列必须按 pyecharts 里 API 要求的顺序 - [开盘价, 收盘价, 最低价, 最高价]。



第 6 -7 行创建 Kline 对象 (标题放左边)，并添加 x 轴数据、y 轴数据和“允许横轴拉伸”。



第 10 -12 行创建 Line 对象 (标题为 SPX，位置离顶 55%) 并起名为 line1，再添加若干属性，比如 y 轴范围、标识最小值、允许横轴拉伸。最关键的是 datazoom_xaxis_index=[2,1,0]，就说伸缩功能控制三个轴，AAPL 一个，SPX 一个，VIX 一个。这样拉伸 x 轴三幅子图可以同时动，非常酷！



第 15 -18 行创建 Line 对象 (标题为 VIX，位置离顶 75%) 并起名为 line2，再添加若干属性。



第 20 行创建 Grid 对象，宽 1000，高 600 (这些数值是不断尝试看效果设置的)。



第 20-23 行将三幅图加在 Grid 中，关键点是如何设置里面的 grid_top 和 grid_bottom 里的百分数而使得图看起来好看，这个没有标准的，不停地尝试到你最终满意为止。本例中 AAPL 占了 5% 到 50% 的位置，SPX 占了 55% 到 70% 的位置，VIX 占了 75% 到 90% 的位置 (还有 10% 位置留给了拉缩轴)。



第 24 行如果被运行，该动态图被生成到 APPL&VIX.html 网页文件里；如果没被运行，该动态图将显示在 Jupyter Notebook 中。

# 时间线轮播图

想画出不同时点 (每月月底) 时五支股票 (苹果、阿里巴巴、京东、脸书和高盛) 交易额 (价格×交易量) 的成分图。



如果用 Matplotlib，是不是要画 n 张饼状图 (pie chart)，其中 n 为月份个数？但是用 PyEcharts 里面的 Timeline 神器，可画出随时间线轮播动态图。一图包含 n 张饼状图。

## 数据准备

In [ ]:
data = stock_data[['Date', 'Symbol', 'Adj Close', 'Volume']]
data.insert(1, 'Year', pd.DatetimeIndex(data['Date']).year)
data.insert(2, 'Month', pd.DatetimeIndex(data['Date']).month)
data.head(3).append(data.tail(3))

- 只保留 'Date', 'Symbol', 'Adj Close' 和 ‘Volume‘，最后两列是股票的收盘价和交易量 (用于计算交易额)



- 从 ‘Date’ 中获取 ‘Year’ 和 ‘Month’ 的信息并插入表中，用于根据年份和月份来分组。

In [ ]:
# 定义个 print_groups 函数便于打印组的名字和前五行信息
def print_groups(group_obj):
    for name, group in group_obj:
        print(name)
        print(group.head())


grouped = data.groupby(['Symbol', 'Year', 'Month'])
print_groups(grouped)

In [ ]:
# 对分好的组想进行什么操作可以用高阶函数 apply() 来实现
# 按 Symbol, Year 和 Month 获取出来了每个月底的 Adj Close 和 Volume
group = grouped.apply(lambda df: df.iloc[-1, :])
group.head(3).append(group.tail(3))

## 绘图

In [ ]:
attr = data.Symbol.unique()
n_stock = len(attr)

MV = group['Adj Close'] * group['Volume']
value = np.reshape(MV.values, (n_stock, -1))
date = group['Date'].dt.strftime('%d-%b-%Y').unique()

timeline = Timeline(
    is_auto_play=True, timeline_bottom=0, width=800, height=400)

for i, d in enumerate(date):
    pie = Pie('Pie Chart', background_color='white', title_pos='left')
    pie.add('',attr,value[:, i],is_label_show=True,radius=[30, 65],
        legend_pos='right',legend_orient='vertical')
    timeline.add(pie.d)

timeline
# timeline.render(path=u'Pie.html')


第 1-2 行获取 5 个股票代号，并且存储股票个数。



第 4 行计算交易额，即收盘价和交易量的乘积。



第 5 行将一维的交易额数组重塑成二维数组，每一行代表一个股票的月末交易额数据。第 6 行获取日期。



第 7-8 行创建 Timeline 对象，允许自动播放，时间线放在底部，宽 800，高 400。



第 11-15 行遍历每个日期，即在每个月末做三件事



创建一个 Pie 对象，设置标题、背景、以及标题位置。



在 Pie 上添加属性名称 (股票代号)、属性对应值 (交易额)、展示标签 (是)、饼半径 (内半径 30 外半径 60 因此是个空心饼状图)、图例位置 (右)、图例排序 (竖直)。



在 Timeline 上添加 Pie 和日期。



第 18 行如果被运行，该动态图被生成到 Pie.html 网页文件里；如果没被运行，该动态图将显示在 Jupyter Notebook 中。

# 词云

In [ ]:
from pyecharts import WordClound

attr=['北京','上海','深圳','广州','杭州','美国','新加坡','武汉','成都','西安',
      '南京','天津','重庆','长沙','苏州','郑州','厦门','香港','合肥','澳大利亚',
     '大连','青岛','济南','沈阳','哈尔滨','长春','福州','英国','加拿大','佛山',
     '珠海','南昌','宁波','东莞','日本','昆明','石家庄','南宁','无锡','德国',
     '太原','贵阳','廊坊','烟台','唐山','兰州','常州','中山','温州','保定']
count=[2058,1695,923,605,568,446,418,364,357,346,
       329,153,130,126,120,116,111,111,100,99,
       94,93,86,79,79,69,67,66,61,59,
       57,54,53,51,43,41,41,41,38,38,
       34,32,30,26,23,23,23,22,22,20]

wordcloud=WordCloud('用户分析',width=800,height=500)
wordcloud.add('',attr,count,shape='star',word_size_range=[20,100])
# wordcloud.render(path=u`'WordCloud.hetml')

第 1 行引用词云对象 WordCloud。



第 3-7 行列出 50 个地名词，第 8-9 行列出相对应的用户数。



第 14 行调用 WordCloud 对象，并设置标题、图宽和图高。



第 15 行添加地名词 attr、用户数 count、形状为星形、词大小范围 20 到 100 之间。假如你不喜欢星形，还可设为圆形，看个人喜好。